In [1]:
import pandas as pd
import numpy as np

In [8]:
df1 = pd.read_csv('data/NoblesLLM_Classes.csv')
df1.head()

,Internal Class ID,Course,COURSE: Subject,Description,Teacher ID,School Year,Class ID
0,33991,Wrestling JV,Athletics,Wrestling JV,24243.0,18-19,WrstJV.W.2018.US
1,36257,Alpine Skiing JV,Athletics,Alpine Skiing JV,24944.0,18-19,SkiJV.W.2018
2,33938,Football Boys Varsity,Athletics,Football Boys Varsity,225027.0,18-19,FblBV.F.2018.US
3,33939,Football Boys Junior,Athletics,Football Boys Junior,23171.0,18-19,FblBJV.F.2018.US
4,33940,Soccer Boys Varsity,Athletics,Soccer Boys Varsity,24959.0,18-19,SocBV.F.2018.US


In [5]:
df2 = pd.read_csv('data/NoblesLLM_Enrollments.csv')
df2.head()

,Enrollment ID,Internal Class ID,STUDENT: Person ID
0,40330,34775.0,10011
1,45862,34775.0,10014
2,56419,38561.0,10022
3,45616,34775.0,10045
4,56531,38561.0,10045


In [6]:
df3 = pd.read_csv('data/NoblesLLM_People.csv')
df3.head()

,Person ID,Gender,Current Grade,City,Roles
0,10001,Female,NaN,Raynham,"Alum '18, Vol"
1,10002,Female,NaN,Wayland,"Alum '19, Sister of Alum"
2,10003,Male,NaN,Brookline,"Alum '18, Vol, Brother of Alum"
3,10004,Female,NaN,Wellesley Hills,"Alum '18, Vol, Sibling"
4,10005,Female,NaN,Dedham,"Alum '18, Vol, Sister of Alum"


In [10]:
# First merge Enrollments with Classes
df_class_enroll = pd.merge(
    left=df2,  # Enrollments
    right=df1,  # Classes
    on='Internal Class ID',
    how='left'
)
df_class_enroll.head()

,Enrollment ID,Internal Class ID,STUDENT: Person ID,Course,COURSE: Subject,Description,Teacher ID,School Year,Class ID
0,40330,34775.0,10011,Journalism I,US: English,Journalism - 1,26540.0,18-19,101001-1
1,45862,34775.0,10014,Journalism I,US: English,Journalism - 1,26540.0,18-19,101001-1
2,56419,38561.0,10022,Journalism I,US: English,Journalism - 1,20009.0,19-20,101001-1
3,45616,34775.0,10045,Journalism I,US: English,Journalism - 1,26540.0,18-19,101001-1
4,56531,38561.0,10045,Journalism I,US: English,Journalism - 1,20009.0,19-20,101001-1


In [11]:
# Then merge with People data
final_df = pd.merge(
    left=df_class_enroll,
    right=df3,  # People
    left_on='STUDENT: Person ID',
    right_on='Person ID',
    how='left'
)
final_df.head()

,Enrollment ID,Internal Class ID,STUDENT: Person ID,Course,COURSE: Subject,Description,Teacher ID,School Year,Class ID,Person ID,Gender,Current Grade,City,Roles
0,40330,34775.0,10011,Journalism I,US: English,Journalism - 1,26540.0,18-19,101001-1,10011.0,Female,NaN,Boston,"Alum '19, Sister of Alum"
1,45862,34775.0,10014,Journalism I,US: English,Journalism - 1,26540.0,18-19,101001-1,10014.0,Female,NaN,Milton,Alum '19
2,56419,38561.0,10022,Journalism I,US: English,Journalism - 1,20009.0,19-20,101001-1,10022.0,Female,NaN,Boston,"Alum '20, Vol"
3,45616,34775.0,10045,Journalism I,US: English,Journalism - 1,26540.0,18-19,101001-1,10045.0,Male,NaN,North Attleboro,"Alum '20, Vol, Brother of Alum"
4,56531,38561.0,10045,Journalism I,US: English,Journalism - 1,20009.0,19-20,101001-1,10045.0,Male,NaN,North Attleboro,"Alum '20, Vol, Brother of Alum"


In [15]:
# First merge Enrollments with Classes to get complete class information
df_class_enroll = pd.merge(
    left=df2,  # Enrollments
    right=df1,  # Classes
    on='Internal Class ID',
    how='left'
)

# Group by student and aggregate class information into a list
class_info = df_class_enroll.groupby('STUDENT: Person ID').apply(
    lambda x: x[['Course', 'COURSE: Subject', 'Internal Class ID', 'School Year']].to_dict('records')
).reset_index(name='Classes')

# Now merge with student information
final_df = pd.merge(
    left=class_info,
    right=df3,  # People
    left_on='STUDENT: Person ID',
    right_on='Person ID',
    how='left'
)

# Display the first row to see the structure
# print(final_df.iloc[0])
# print(final_df.head())
final_df.head()

/var/folders/80/ghh7mq_5367crl38_k3_k0lm0000gp/T/ipykernel_88984/3110599329.py:10: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  class_info = df_class_enroll.groupby('STUDENT: Person ID').apply(


,STUDENT: Person ID,Classes,Person ID,Gender,Current Grade,City,Roles
0,0,"[{'Course': nan, 'COURSE: Subject': nan, 'Inte...",NaN,NaN,NaN,NaN,NaN
1,10001,"[{'Course': nan, 'COURSE: Subject': nan, 'Inte...",10001.0,Female,NaN,Raynham,"Alum '18, Vol"
2,10002,"[{'Course': 'Ethics and Literature', 'COURSE: ...",10002.0,Female,NaN,Wayland,"Alum '19, Sister of Alum"
3,10003,"[{'Course': nan, 'COURSE: Subject': nan, 'Inte...",10003.0,Male,NaN,Brookline,"Alum '18, Vol, Brother of Alum"
4,10004,"[{'Course': nan, 'COURSE: Subject': nan, 'Inte...",10004.0,Female,NaN,Wellesley Hills,"Alum '18, Vol, Sibling"


In [28]:
import json

# Get the first student's record and convert to dictionary
student_record = final_df.iloc[10].to_dict()

# Print formatted JSON
print(json.dumps(student_record, indent=2))

{
  "STUDENT: Person ID": 10010,
  "Classes": [
    {
      "Course": "Madness in Literature",
      "COURSE: Subject": "US: English",
      "Internal Class ID": 34779.0,
      "School Year": "18-19"
    },
    {
      "Course": "Chimera: Mythology, Classical Lit",
      "COURSE: Subject": "US: English",
      "Internal Class ID": 34786.0,
      "School Year": "18-19"
    },
    {
      "Course": "Creative Writing",
      "COURSE: Subject": "US: English",
      "Internal Class ID": 34788.0,
      "School Year": "18-19"
    },
    {
      "Course": "AP Statistics",
      "COURSE: Subject": "US: Mathematics",
      "Internal Class ID": 34844.0,
      "School Year": "18-19"
    },
    {
      "Course": "Robotics",
      "COURSE: Subject": "US: Science",
      "Internal Class ID": 34892.0,
      "School Year": "18-19"
    },
    {
      "Course": "American History and Hollywood Film",
      "COURSE: Subject": "US: History and Social Science",
      "Internal Class ID": 34951.0,
      "Scho

In [23]:
from faker import Faker
import random
import numpy as np

# Set up Faker with a fixed seed for reproducibility
fake = Faker()
Faker.seed(12345)

# Create a dictionary mapping student IDs to fake names
# Only include non-null IDs
student_ids = final_df['Person ID'].dropna().unique()
name_mapping = {}
for student_id in student_ids:
    name_mapping[student_id] = {
        'first_name': fake.first_name(),
        'last_name': fake.last_name()
    }

# Function to replace student ID with name in our records
def add_names_to_record(record):
    student_id = record['Person ID']
    record_copy = record.copy()
    
    # Check if student_id is valid (not NaN)
    if pd.notna(student_id) and student_id in name_mapping:
        record_copy['first_name'] = name_mapping[student_id]['first_name']
        record_copy['last_name'] = name_mapping[student_id]['last_name']
    else:
        # Handle missing IDs by generating a random name
        record_copy['first_name'] = fake.first_name()
        record_copy['last_name'] = fake.last_name()
    
    # Remove the ID fields (optional)
    del record_copy['Person ID']
    del record_copy['STUDENT: Person ID']
    
    return record_copy

# Create new dataframe with names
named_df = final_df.apply(lambda x: add_names_to_record(x), axis=1)

# Display an example student
print(json.dumps(named_df.iloc[10].to_dict(), indent=2))

# Optional: Display how many records had missing IDs
missing_ids = final_df['Person ID'].isna().sum()
print(f"\nNumber of records with missing IDs: {missing_ids}")

{
  "Classes": [
    {
      "Course": "Madness in Literature",
      "COURSE: Subject": "US: English",
      "Internal Class ID": 34779.0,
      "School Year": "18-19"
    },
    {
      "Course": "Chimera: Mythology, Classical Lit",
      "COURSE: Subject": "US: English",
      "Internal Class ID": 34786.0,
      "School Year": "18-19"
    },
    {
      "Course": "Creative Writing",
      "COURSE: Subject": "US: English",
      "Internal Class ID": 34788.0,
      "School Year": "18-19"
    },
    {
      "Course": "AP Statistics",
      "COURSE: Subject": "US: Mathematics",
      "Internal Class ID": 34844.0,
      "School Year": "18-19"
    },
    {
      "Course": "Robotics",
      "COURSE: Subject": "US: Science",
      "Internal Class ID": 34892.0,
      "School Year": "18-19"
    },
    {
      "Course": "American History and Hollywood Film",
      "COURSE: Subject": "US: History and Social Science",
      "Internal Class ID": 34951.0,
      "School Year": "18-19"
    },
    {


In [29]:
import pandas as pd
import numpy as np
from faker import Faker
import json
import random

# Set up Faker
fake = Faker()
Faker.seed(12345)

# Read data
df1 = pd.read_csv('data/NoblesLLM_Classes.csv')
df2 = pd.read_csv('data/NoblesLLM_Enrollments.csv')
df3 = pd.read_csv('data/NoblesLLM_People.csv')

# Filter out students with NaN grades
df3 = df3.dropna(subset=['Current Grade'])

# First merge Enrollments with Classes to get complete class information
df_class_enroll = pd.merge(
    left=df2,  # Enrollments
    right=df1,  # Classes
    on='Internal Class ID',
    how='left'
)

# Filter out NaN courses
df_class_enroll = df_class_enroll.dropna(subset=['Course'])

# Group by student and aggregate class information
class_info = df_class_enroll.groupby('STUDENT: Person ID').apply(
    lambda x: x[['Course', 'COURSE: Subject', 'Internal Class ID', 'School Year']].to_dict('records')
).reset_index(name='Classes')

# Merge with filtered student information
final_df = pd.merge(
    left=class_info,
    right=df3,  # Filtered People data
    left_on='STUDENT: Person ID',
    right_on='Person ID',
    how='inner'  # Changed to inner join to only keep matched students
)

# Create name mapping
student_ids = final_df['Person ID'].dropna().unique()
name_mapping = {
    student_id: {
        'first_name': fake.first_name(),
        'last_name': fake.last_name()
    } for student_id in student_ids
}

# Function to replace student ID with name
def add_names_to_record(record):
    student_id = record['Person ID']
    record_copy = record.copy()
    
    if pd.notna(student_id) and student_id in name_mapping:
        record_copy['first_name'] = name_mapping[student_id]['first_name']
        record_copy['last_name'] = name_mapping[student_id]['last_name']
    else:
        record_copy['first_name'] = fake.first_name()
        record_copy['last_name'] = fake.last_name()
    
    del record_copy['Person ID']
    del record_copy['STUDENT: Person ID']
    
    return record_copy

# Create final dataframe with names
named_df = final_df.apply(lambda x: add_names_to_record(x), axis=1)

# Display an example student
print(json.dumps(named_df.iloc[0].to_dict(), indent=2))

{
  "Classes": [
    {
      "Course": "Satire and Humor",
      "COURSE: Subject": "US: English",
      "Internal Class ID": 49407.0,
      "School Year": "24-25"
    },
    {
      "Course": "Page and Stage",
      "COURSE: Subject": "US: English",
      "Internal Class ID": 50260.0,
      "School Year": "24-25"
    },
    {
      "Course": "English II",
      "COURSE: Subject": "US: English",
      "Internal Class ID": 47584.0,
      "School Year": "23-24"
    },
    {
      "Course": "English III",
      "COURSE: Subject": "US: English",
      "Internal Class ID": 44541.0,
      "School Year": "22-23"
    },
    {
      "Course": "English IV",
      "COURSE: Subject": "US: English",
      "Internal Class ID": 42900.0,
      "School Year": "21-22"
    },
    {
      "Course": "English V",
      "COURSE: Subject": "MS: English MS",
      "Internal Class ID": 41254.0,
      "School Year": "20-21"
    },
    {
      "Course": "English via Latin",
      "COURSE: Subject": "MS: English M

/var/folders/80/ghh7mq_5367crl38_k3_k0lm0000gp/T/ipykernel_88984/2754633183.py:31: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  class_info = df_class_enroll.groupby('STUDENT: Person ID').apply(


In [32]:
# Take a random sample of 100 students
sampled_df = named_df.sample(n=2, random_state=42)  # random_state for reproducibility

# Export to CSV
sampled_df.to_csv('data/nobles_sample_2.csv', index=False)

# Print shape to confirm
print(f"Original dataset shape: {named_df.shape}")
print(f"Sampled dataset shape: {sampled_df.shape}")

# Display first few rows to verify content
print("\nFirst few rows of sampled dataset:")
print(sampled_df.head())

Original dataset shape: (632, 7)
Sampled dataset shape: (2, 7)

First few rows of sampled dataset:
                                               Classes  Gender Current Grade  \
328  [{'Course': 'English II', 'COURSE: Subject': '...  Female      Grade 11   
247  [{'Course': 'English IV', 'COURSE: Subject': '...  Female       Grade 9   

             City           Roles first_name last_name  
328        Weston  Student (II-D)      Corey  Anderson  
247  West Roxbury  Student (IV-D)      Linda     Clark  
